## Extração de dados sobre o coronavírus

### Dados da Secretaria de Saúde de Pernambuco


In [1]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import tabula

### Pegando informações da página

Primeiro vamos pegar o código da página da Secretaria de Saúde

No caso da Secretaria de Pernambuco, os links de interesse ficam na
tag **h1**. Por isso iremos buscar por elas no documento.

In [2]:
url = 'https://www.cievspe.com/novo-coronavirus-2019-ncov'

html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')


h1 = soup.find_all('h1', 'font_0')

### Fazendo download do arquivo

Após criar uma lista com todos os H1s do site, iteramos procurando 
o conteúdo da tag **span**, pois nela estão os nomes dos arquivos que
queremos baixar.

Ao encontrar o arquivo **Informe Epidemiológico Coronavírus (COVID-19)**, 
que possui uma numeração com a atualização, fazemos o dowload do 
PDF que será tratado posteriormente.

In [4]:
for span in h1:
    if span.find('span') is not None:
        if 'Informe Epidemiológico Coronavírus (COVID-19)' in span.text:
            print(span.text)
            print(span.find('a')['href'])
            filename = Path('covid19_pe.pdf')
            url = span.find('a')['href']
            response = requests.get(url)
            filename.write_bytes(response.content)


20/03/2020 Informe Epidemiológico Coronavírus (COVID-19) | Nº 19 - Pernambuco
https://12ad4c92-89c7-4218-9e11-0ee136fa4b92.filesusr.com/ugd/3293a8_bb922dee19844ac7b24fe5649aa692a0.pdf


 Abaixo está a conversão da tabela em PDF para o formato CSV.
 
 * Tabula é uma biblioteca que lê a tabela magicamente 
 * 4 é a página que tem a tabela de interesse
 

In [10]:
tabula.convert_into("covid19_pe.pdf", "covid19_pe.csv", output_format="csv", pages=4)

df = pd.read_csv('covid19_pe.csv')

### Tratando os dados da tabela

Após criar o CSV com os dados, precisamos trocar os nomes das colunas
de números (1 a 6) para algo mais explícito. Após isso, salvamos as
informações no CSV novamente.

In [11]:
df = df.T.reset_index(drop=True).T
df = df.drop([0,1])
df = df.iloc[ :,1:]

df = df.rename(columns={1:'MUNICÍPIO', 2:'SUSPEITO', 3:'PROVÁVEL', 4:'CONFIRMADO', 5:'DESCARTADO', 6:'TOTAL'})

cols = ['SUSPEITO', 'PROVÁVEL', 'CONFIRMADO', 'DESCARTADO', 'TOTAL']
for col in cols:
    df[col] = df[col].map(lambda x: str(x).lstrip('*').rstrip('*'))
    
df['MUNICÍPIO'] = df['MUNICÍPIO'].map(lambda x: str(x).replace('\n', ' '))
df = df.sort_values(['MUNICÍPIO'], ascending=1)
df = df.reset_index(drop=True)


In [12]:
df.to_csv('covid19_pe.csv')

df.head()

,MUNICÍPIO,SUSPEITO,PROVÁVEL,CONFIRMADO,DESCARTADO,TOTAL
0,Abreu e Lima,6,0,0,1,7
1,Barreiros,1,0,0,1,2
2,Belo Jardim,0,0,1,0,1
3,Bezerros,1,0,0,0,1
4,Cabo de Santo Agostinho,4,0,0,0,4
